In [1]:
## import libraries needed to load the model and other functions

import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
## First load the trainedmodel

model = load_model("/Users/muralis/AIML/data/muralitest.keras")

In [15]:
## Read the encoders and scalers stored in the file

with open("/Users/muralis/AIML/data/genderencoder.pk1",'rb') as file:
    labelencoder_gen = pickle.load(file)
file.close()

with open("/Users/muralis/AIML/data/geoencoder.pk1",'rb') as file:
    geo_encoder = pickle.load(file)
file.close()

with open("/Users/muralis/AIML/data/scalerdump.pk1",'rb') as file:
    scaler = pickle.load(file)
file.close()

In [49]:
## Define the input data

# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 30,
    'Tenure': 3,
    'Balance': 6000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [50]:
## change the gender in the input data

##convert the input data into a data frame

input_df = pd.DataFrame([input_data], columns= ["CreditScore","Geography","Gender","Age","Tenure","Balance","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary"])
input_df["Gender"] = labelencoder_gen.transform(input_df["Gender"])
print(input_df.head())

   CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          600    France       1  ...          1               1            50000

[1 rows x 10 columns]


In [51]:
##encode the Geo for the input data
geo_encoded_val = geo_encoder.transform([input_df["Geography"]])
type(geo_encoded_val)
geo_encoded_df = pd.DataFrame(geo_encoded_val.toarray(),columns=geo_encoder.get_feature_names_out(["Geography"]))
print(geo_encoded_df)


   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0


/Users/muralis/AIML/deepl/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [52]:
## drop the geography column form the df and add the geo_encoded_matrix

data_to_predict = pd.concat([input_df.drop(["Geography"],axis=1), geo_encoded_df], axis=1)

In [54]:
## Use the scalar function on top of it now to normalize the data

data_to_predict_scaled = scaler.transform(data_to_predict)
print(data_to_predict_scaled)

[[-0.52623557  0.91287093 -0.84683466 -0.7014635  -1.13089089  0.76799334
   0.64663354  0.97073117 -0.85513815  1.01896773 -0.57035183 -0.59881865]]


In [55]:
##predict the outcome

prediction = model.predict(data_to_predict_scaled)
print(prediction)
print(prediction[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[[0.00221969]]
0.0022196874


In [ ]:
if(prediction[0][0],0.5):
    print('The probability of customer leaving us is very low')
else:
    print("The probability of customer leaving is very high")